API limitations:
 - 50 series allowed per pull
 - 20 years allowed per pull
 - approx 5000 PPI series with some series having data starting in the 1960s means approx 300 pulls required - 100 for all series, * 3 for 20 year selections out to the 60s
 - daily limit of 500 pulls means this can only be ran once a day

In [2]:
import requests
import json
import pandas as pd
from bls_definitions import bls_ppi_codes
import datetime
import os

In [4]:
key = os.environ['BLS_API_KEY']
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

# Looping all codes in sets of 50 over three 20 year periods

In [ ]:
# create date ranges to loop on
endyear = datetime.datetime.now().year

date_ranges = {0 : (endyear-19, endyear),
               1 : (endyear-39, endyear-20),
               2 : (endyear-59, endyear-40)}

In [ ]:
bls_ppi_code_segments = [list(bls_ppi_codes.keys())[x:x+50]for x in range(0, len(list(bls_ppi_codes.keys())), 50)]

In [ ]:
master_dfs = {}
master_missing_series = {}
segment_no = 0

# loop over sets of 50 codes
for i in bls_ppi_code_segments:
    dfs_diff_periods = {}
    missing_series = {}
    for j in date_ranges:
        # create variatble selection dictionary
        selection_dict = {"seriesid": i, "startyear":date_ranges[j][0], "endyear":date_ranges[j][1], "registrationkey" : key}
        selection = json.dumps(selection_dict)

        # send request to BLS
        p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=selection, headers=headers)
        p = p.json()['Results']['series']

        # create dictionaries to hold values and index
        data = {}
        index = {}

        # save time series and index for each variable
        for k in p:
            data[k['seriesID']] = pd.Series([l['value'] for l in k['data']]).astype(float).iloc[::-1].values
            index[k['seriesID']] = [f"{l['year']}-{l['period'][1:]}-01" for l in k['data']][::-1]

        # identify where codes are missing in each period window
        missing_codes = []
        for k in p:
            if len(k['data']) == 0:
                missing_codes.append(k['seriesID'])
        missing_series[j] = missing_codes

        # convert series into dataframes and then combine into one dataframe for the period window
        dfs = {}
        for k in data.keys():
            dfs[k] = pd.DataFrame(data = data[k], index = pd.to_datetime(index[k]))

        final = pd.concat(dfs,axis=1)
        final.columns = final.columns.get_level_values(0)

        dfs_diff_periods[j] = final
    
    # combine dfs of series into one dataframe
    segment_df = pd.concat(dfs_diff_periods)
    segment_df.index = segment_df.index.droplevel(0)
    segment_df = segment_df.sort_index()

    # export segment df to a parquet file format for reduced file size storage
    # once API is run the dat is then saved - no need to rerun every time
    segment_df.to_parquet('C:\\DSWG_PPI\\api_pulls\\'+str(segment_no)+'api.gzip', compression='gzip')
    
    # save segment df to master dictionary
    master_dfs[segment_no] = segment_df

    # identify codes with no values across all periods
    missing_sets = {}
    for j in missing_series.keys():
        missing_sets[j] = set(missing_series[j])

    variables_no_data = missing_sets[0].intersection(missing_sets[1]).intersection(missing_sets[2])

    master_missing_series[segment_no] = variables_no_data

    print(segment_no)
    segment_no += 1
    

C:\Users\PETERC~1\AppData\Local\Temp/ipykernel_3688/3364293840.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data[k['seriesID']] = pd.Series([l['value'] for l in k['data']]).astype(float).iloc[::-1].values


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


KeyError: 'series'

In [ ]:
stop

In [ ]:
# # merge master dataframes into 
# all_ppi = pd.concat(master_dfs)
# all_ppi.index = all_ppi.index.droplevel(0)
# all_ppi = all_ppi.sort_index()

# # remove duplicate index values
# all_ppi = all_ppi[~all_ppi.index.duplicated(keep='first')]

In [ ]:
# all_ppi.to_csv('test.csv')

In [ ]:
all_ppi = pd.DataFrame(index=pd.date_range(start='1/1/1960', end =  str(datetime.datetime.now().month) + '/1/' + str(datetime.datetime.now().year), freq='ms'))

MemoryError: Unable to allocate 14.5 TiB for an array with shape (1995926400001,) and data type int64

In [ ]:
master_dfs

In [ ]:
for i in master_dfs.keys():
    for j in master_dfs[i].columns:
        all_ppi[j] = master_dfs[i][j]

In [ ]:
all_ppi.to_csv('test.csv')

In [ ]:
all_ppi